In [ ]:
import re
import nltk

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
5
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.preprocessing import LabelEncoder
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


## 1. Генерирование русских имен при помощи RNN

Датасет: https://disk.yandex.ru/i/2yt18jHUgVEoIw

1.1 На основе файла name_rus.txt создайте датасет.
  * Учтите, что имена могут иметь различную длину
  * Добавьте 4 специальных токена:
    * `<PAD>` для дополнения последовательности до нужной длины;
    * `<UNK>` для корректной обработки ранее не встречавшихся токенов;
    * `<SOS>` для обозначения начала последовательности;
    * `<EOS>` для обозначения конца последовательности.
  * Преобразовывайте строку в последовательность индексов с учетом следующих замечаний:
    * в начало последовательности добавьте токен `<SOS>`;
    * в конец последовательности добавьте токен `<EOS>` и, при необходимости, несколько токенов `<PAD>`;
  * `Dataset.__get_item__` возращает две последовательности: последовательность для обучения и правильный ответ.
  
  Пример:
  ```
  s = 'The cat sat on the mat'
  # преобразуем в индексы
  s_idx = [2, 5, 1, 2, 8, 4, 7, 3, 0, 0]
  # получаем x и y (__getitem__)
  x = [2, 5, 1, 2, 8, 4, 7, 3, 0]
  y = [5, 1, 2, 8, 4, 7, 3, 0, 0]
  ```

1.2 Создайте и обучите модель для генерации фамилии.

  * Для преобразования последовательности индексов в последовательность векторов используйте `nn.Embedding`;
  * Используйте рекуррентные слои;
  * Задача ставится как предсказание следующего токена в каждом примере из пакета для каждого момента времени. Т.е. в данный момент времени по текущей подстроке предсказывает следующий символ для данной строки (задача классификации);
  * Примерная схема реализации метода `forward`:
  ```
    input_X: [batch_size x seq_len] -> nn.Embedding -> emb_X: [batch_size x seq_len x embedding_size]
    emb_X: [batch_size x seq_len x embedding_size] -> nn.RNN -> output: [batch_size x seq_len x hidden_size]
    output: [batch_size x seq_len x hidden_size] -> torch.Tensor.reshape -> output: [batch_size * seq_len x hidden_size]
    output: [batch_size * seq_len x hidden_size] -> nn.Linear -> output: [batch_size * seq_len x vocab_size]
  ```

1.3 Напишите функцию, которая генерирует фамилию при помощи обученной модели:
  * Построение начинается с последовательности единичной длины, состоящей из индекса токена `<SOS>`;
  * Начальное скрытое состояние RNN `h_t = None`;
  * В результате прогона последнего токена из построенной последовательности через модель получаете новое скрытое состояние `h_t` и распределение над всеми токенами из словаря;
  * Выбираете 1 токен пропорционально вероятности и добавляете его в последовательность (можно воспользоваться `torch.multinomial`);
  * Повторяете эти действия до тех пор, пока не сгенерирован токен `<EOS>` или не превышена максимальная длина последовательности.

При обучении каждые `k` эпох генерируйте несколько фамилий и выводите их на экран.

In [ ]:
import re
import nltk

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
# df_names = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/name_rus.txt', encoding='cp1251', header=None)
df_names = pd.read_csv('name_rus.txt', encoding='cp1251', header=None)
df_names.columns = ['name']
df_names

,name
0,авдокея
1,авдоким
2,авдоня
3,авдотька
4,авдотьюшка
...,...
1983,ярополк
1984,ярослав
1985,ярослава
1986,ярославка


In [ ]:
class Vocab:
    def __init__(self, data):
        self.idx_to_token, self.token_to_idx = self.build_vocab(data)
        self.vocab_len = len(self.idx_to_token)
        self.max_seq_len = self.calculate_max_seq_len(data)

    def build_vocab(self, data):
        tokens = list()
        tokens.append('<PAD>')
        tokens.append('<UNK>')
        tokens.append('<SOS>')
        tokens.append('<EOS>')

        for name in data:
          for token in name:
            if token not in tokens:
                tokens.append(token.lower())

        idx_to_token = {idx: token for idx, token in enumerate(tokens, start=0)}
        token_to_idx = {token: idx for idx, token in enumerate(tokens, start=0)}

        return idx_to_token, token_to_idx

    def calculate_max_seq_len(self, data):
        return max(len(name) + 2 for name in data)

    def process_name(self, name):
        indices = [self.token_to_idx['<SOS>']] + [self.token_to_idx.get(char, self.token_to_idx['<UNK>']) for char in name.lower()] + [self.token_to_idx['<EOS>']]
        pad_length = self.max_seq_len - len(indices)
        indices += [self.token_to_idx['<PAD>']] * pad_length
        return indices

    def indices_to_chars(self, indices):
        return [self.idx_to_token[idx] for idx in indices]

    def chars_to_indices(self, chars):
        return [self.token_to_idx.get(char, self.token_to_idx['<UNK>']) for char in chars]

In [ ]:
vocab = Vocab(df_names['name'])
vocab.max_seq_len

15

In [ ]:
vocab.idx_to_token

{0: '<PAD>',
 1: '<UNK>',
 2: '<SOS>',
 3: '<EOS>',
 4: 'а',
 5: 'в',
 6: 'д',
 7: 'о',
 8: 'к',
 9: 'е',
 10: 'я',
 11: 'и',
 12: 'м',
 13: 'н',
 14: 'т',
 15: 'ь',
 16: 'ю',
 17: 'ш',
 18: 'х',
 19: 'у',
 20: 'л',
 21: 'с',
 22: 'г',
 23: 'р',
 24: 'ф',
 25: 'п',
 26: 'ч',
 27: 'ы',
 28: 'б',
 29: 'й',
 30: 'ж',
 31: 'ц',
 32: 'з',
 33: 'э'}

In [ ]:
import torch
from torch.utils.data import Dataset

class NameDataset(Dataset):
    def __init__(self, data, vocab):
        self.data = data
        self.vocab = vocab

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        name = self.data[index]
        indices = self.vocab.process_name(name)
        x = torch.tensor(indices[:-1], dtype=torch.long)
        y = torch.tensor(indices[1:], dtype=torch.long)
        return x, y


In [ ]:
name_dataset = NameDataset(df_names['name'], vocab)

print(df_names.iloc[0, 0])
print(name_dataset[0])

авдокея
(tensor([ 2,  4,  5,  6,  7,  8,  9, 10,  3,  0,  0,  0,  0,  0]), tensor([ 4,  5,  6,  7,  8,  9, 10,  3,  0,  0,  0,  0,  0,  0]))


In [ ]:
class NameGenerator(nn.Module):
    def __init__(self, vocab_size, embedding_size, hidden_size, num_layers=2):
        super(NameGenerator, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.lstm = nn.LSTM(embedding_size, hidden_size, num_layers=num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, vocab_size)

    def forward(self, x):
        x = self.embedding(x)
        output, _ = self.lstm(x)
        output = self.fc1(output)
        return output

vocab_size, embedding_size, hidden_size = vocab.vocab_len, 10, 64
model = NameGenerator(vocab_size, embedding_size, hidden_size)

vocab = Vocab(df_names['name'])
train_dataset = NameDataset(data=df_names['name'], vocab=vocab)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.008)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [ ]:
param_num = sum(p.numel() for p in model.parameters())
print("Количество параметров:", param_num)

Количество параметров: 55286


In [ ]:
model

NameGenerator(
  (embedding): Embedding(34, 10)
  (lstm): LSTM(10, 64, num_layers=2, batch_first=True)
  (fc1): Linear(in_features=64, out_features=34, bias=True)
)

In [ ]:
def train(model, train_dl, optimizer, criterion, vocab):
    model.train()

    total_loss = 0.0
    total_samples = 0.0
    correct_samples = 0.0

    for batch in train_dl:
        inputs, targets = batch

        outputs = model(inputs)
        targets = targets.view(-1)

        loss = criterion(outputs.view(-1, vocab.vocab_len), targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss
        total_samples += targets.shape[0]

    perplexity = 2 ** (total_loss / len(train_dl))
    avg_loss = total_loss / len(train_dl)

    return avg_loss, perplexity

In [ ]:
def generate_names(model, vocab, max_length=20, num_names=20):
    model.eval()

    names_list = []

    for _ in range(num_names):
        current_token = torch.tensor([[vocab.token_to_idx['<SOS>']]], dtype=torch.long)
        generated_name = []

        for _ in range(max_length):
            output = model(current_token)

            probabilities = torch.softmax(output[:, -1, :], dim=1)

            next_token = torch.multinomial(probabilities, 1)

            next_token = int(next_token.item())

            if next_token == vocab.token_to_idx['<EOS>']:
                break

            if vocab.idx_to_token[next_token] not in ['<PAD>', '<UNK>', '<SOS>', '<EOS>']:
                generated_name.append(vocab.idx_to_token[next_token])

            current_token = torch.cat([current_token, torch.tensor([[next_token]], dtype=torch.long)], dim=1)

        names_list.append(''.join(generated_name).capitalize())

    return ', '.join(names_list)

In [ ]:
losses_train, losses_test = [], []
for epoch in range(50):
    total_loss_train, perplexity = train(model, train_loader, optimizer, criterion, vocab)
    losses_train.append(total_loss_train.item())
    # total_loss_test, test_accuracy = test(model, test_loader, criterion, vocab)
    # losses_test.append(total_loss_test.item())
    print(f"epoch [{epoch+1}]: Average Loss: [{total_loss_train:.4f}], Perplexity: [{perplexity:.4f}],")
    if (epoch + 1) % 1 == 0:
        generated_name = generate_names(model, vocab)
        print(f"Generated Name: {generated_name}")

epoch [1]: Average Loss: [1.9725], Perplexity: [3.9245],
Generated Name: Нлиемнаяя, Жмлао, Талнмеаеан, Ллеуах, Цж, Япул, Лбхуьяв, Иртвешбрн, Хя, Хикакю, Эааи, Тгдяа, Оралаас, Луио, Шш, Яюраа, Гдиоаиар, Синоа, Ыйрряс, Викш
epoch [2]: Average Loss: [1.3967], Perplexity: [2.6330],
Generated Name: Ин, Дитскиьау, Рлмкыиуа, Гиа, Ешникаа, Врейна, Олмьспнаа, Зпшуана, Шеа, Вааяюя, Спуяя, Одоаха, Лгазкка, Арлмтлыраа, Оксчдяь, Алвкса, Хттускнлчй, Руаи, Лансар, Аиаанса
epoch [3]: Average Loss: [1.2402], Perplexity: [2.3624],
Generated Name: Арих, Ирюма, Маисиса, Гетонидя, Етм, Лраьаню, Мвароша, Редя, Дкарю, Штедюяы, Еонч, Евмаликя, Гака, Раикя, Пяраха, Вратас, Ртното, Оухи, Амуфа, Янрунткач
epoch [4]: Average Loss: [1.0913], Perplexity: [2.1306],
Generated Name: Сукаша, Стамя, Паник, Анша, Ансорса, Вука, Аэля, Луня, Местаска, Арлита, Видкя, Икийна, Тарана, Иоша, Таря, Оння, Восин, Сосфуша, Мкалрин, Вадя
epoch [5]: Average Loss: [1.0505], Perplexity: [2.0712],
Generated Name: Алол, Авелан, Пернка, 

In [ ]:
def generate_names(model, vocab, max_length=20, num_names=20):
    model.eval()

    names_list = []

    for _ in range(num_names):
        current_token = torch.tensor([[vocab.token_to_idx['<SOS>']]], dtype=torch.long)
        generated_name = []

        for _ in range(max_length):
            output = model(current_token)

            probabilities = torch.softmax(output[:, -1, :], dim=1)

            next_token = torch.multinomial(probabilities, 1)

            # Convert the sampled token to a Python integer
            next_token = int(next_token.item())

            if next_token == vocab.token_to_idx['<EOS>']:
                break

            # Check if the sampled token is not a special symbol
            if vocab.idx_to_token[next_token] not in ['<PAD>', '<UNK>', '<SOS>', '<EOS>']:
                # Append the sampled token to the generated surname
                generated_name.append(vocab.idx_to_token[next_token])

            current_token = torch.cat([current_token, torch.tensor([[next_token]], dtype=torch.long)], dim=1)

        names_list.append(''.join(generated_name).capitalize())

    return ', '.join(names_list)


    return ', '.join(names_list)
generated_name = generate_names(model, vocab)
print(f"Generated Name: {generated_name}")

Generated Name: Константинка, Килина, Сенюра, Валюха, Элизуня, Александра, Яда, Игорыч, Танюша, Килинка, Петруня, Захарий, Лавреныч, Емиля, Аруся, Веруся, Эмиля, Андрейка, Тонюся, Владислава


## 2. Генерирование текста при помощи RNN

2.1 Скачайте из интернета какое-нибудь художественное произведение
  * Выбирайте достаточно крупное произведение, чтобы модель лучше обучалась;

2.2 На основе выбранного произведения создайте датасет.

Отличия от задачи 1:
  * Токены <SOS>, `<EOS>` и `<UNK>` можно не добавлять;
  * При создании датасета текст необходимо предварительно разбить на части. Выберите желаемую длину последовательности `seq_len` и разбейте текст на построки длины `seq_len` (можно без перекрытия, можно с небольшим перекрытием).

2.3 Создайте и обучите модель для генерации текста
  * Задача ставится точно так же как в 1.2;
  * При необходимости можете применить:
    * двухуровневые рекуррентные слои (`num_layers`=2)
    * [обрезку градиентов](https://pytorch.org/docs/stable/generated/torch.nn.utils.clip_grad_norm_.html)

2.4 Напишите функцию, которая генерирует фрагмент текста при помощи обученной модели
  * Процесс генерации начинается с небольшого фрагмента текста `prime`, выбранного вами (1-2 слова)
  * Сначала вы пропускаете через модель токены из `prime` и генерируете на их основе скрытое состояние рекуррентного слоя `h_t`;
  * После этого вы генерируете строку нужной длины аналогично 1.3


**Генерация на основе слов**

In [ ]:
with open("Толстой Лев. Война и мир. Шедевр мировой литературы в одном томе - royallib.com.txt", "r", encoding="cp1251") as file:
    text = file.read().replace('\n', ' ').lower()
    text = re.sub(r'[^а-яА-ЯёЁ0-9]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
text[:1000]

'приятного чтения лев толстой война и мир шедевр мировой литературы в одном томе том первый часть первая 234 поместья 233 233 234 234 мой верный раб 1 ну князь генуя и лукка поместья фамилии бонапарте нет я вам вперед говорю если вы мне не скажете что у нас война если вы еще позволите себе защищать все гадости все ужасы этого антихриста право я верю что он антихрист я вас больше не знаю вы уж не друг мой вы уж не мой верный раб как вы говорите франц в дальнейшем переводы с французского не оговариваются здесь и далее все переводы кроме специально оговоренных принадлежат л н толстому ред ну здравствуйте здравствуйте 2 я вижу что я вас пугаю садитесь и рассказывайте так говорила в июле 1805 года известная анна павловна шерер фрейлина и приближенная императрицы марии феодоровны встречая важного и чиновного князя василия первого приехавшего на ее вечер анна павловна кашляла несколько дней у нее был грипп как она говорила грипп был тогда новое слово употреблявшееся только редкими в записочка

In [ ]:
len(text.split())

464110

In [ ]:
text = text.split()[150000:200000]

In [ ]:
len(text)

50000

In [ ]:
seq_len = 50
step_size = 1

def create_dataset(text, seq_len, step_size):
    sequences = []
    for i in range(0, len(text) - seq_len + 1, step_size):
        seq = text[i : i + seq_len]
        sequences.append(seq)

    df = pd.DataFrame({'sequences': sequences})
    return df

df_sequences = create_dataset(text, seq_len, step_size)
df_sequences

,sequences
0,"[парадной, форме, мимо, которых, ему, надо, бы..."
1,"[форме, мимо, которых, ему, надо, было, пройти..."
2,"[мимо, которых, ему, надо, было, пройти, прокл..."
3,"[которых, ему, надо, было, пройти, проклиная, ..."
4,"[ему, надо, было, пройти, проклиная, свою, сме..."
...,...
49946,"[на, другой, день, князь, ни, слова, не, сказа..."
49947,"[другой, день, князь, ни, слова, не, сказал, с..."
49948,"[день, князь, ни, слова, не, сказал, своей, до..."
49949,"[князь, ни, слова, не, сказал, своей, дочери, ..."


In [ ]:
class Vocab:
    def __init__(self, data):
        self.idx_to_token, self.token_to_idx = self.build_vocab(data)
        self.vocab_len = len(self.idx_to_token)
        self.max_seq_len = self.calculate_max_seq_len(data)

    def build_vocab(self, data):
        tokens = list()
        tokens.append('<PAD>')
        tokens.append('<SOS>')


        for seq in data:
          for token in seq:
            if token not in tokens:
                tokens.append(token)

        idx_to_token = {idx: token for idx, token in enumerate(tokens, start=0)}
        token_to_idx = {token: idx for idx, token in enumerate(tokens, start=0)}

        return idx_to_token, token_to_idx

    def calculate_max_seq_len(self, data):
        return max(len(seq) + 2 for seq in data)

    def process_text(self, seq):
        indices = [self.token_to_idx['<SOS>']] + [self.token_to_idx.get(char) for char in seq]
        pad_length = self.max_seq_len - len(indices)
        indices += [self.token_to_idx['<PAD>']] * pad_length
        return indices

    def indices_to_chars(self, indices):
        return [self.idx_to_token[idx] for idx in indices]

    def chars_to_indices(self, chars):
        return [self.token_to_idx.get(char) for char in chars]

vocab = Vocab(df_sequences['sequences'])
vocab.vocab_len

11785

In [ ]:
vocab.idx_to_token

{0: '<PAD>',
 1: '<SOS>',
 2: 'парадной',
 3: 'форме',
 4: 'мимо',
 5: 'которых',
 6: 'ему',
 7: 'надо',
 8: 'было',
 9: 'пройти',
 10: 'проклиная',
 11: 'свою',
 12: 'смелость',
 13: 'замирая',
 14: 'от',
 15: 'мысли',
 16: 'что',
 17: 'всякую',
 18: 'минуту',
 19: 'он',
 20: 'может',
 21: 'встретить',
 22: 'государя',
 23: 'и',
 24: 'при',
 25: 'нем',
 26: 'быть',
 27: 'осрамлен',
 28: 'выслан',
 29: 'под',
 30: 'арест',
 31: 'понимая',
 32: 'вполне',
 33: 'всю',
 34: 'неприличность',
 35: 'своего',
 36: 'поступка',
 37: 'раскаиваясь',
 38: 'в',
 39: 'ростов',
 40: 'опустив',
 41: 'глаза',
 42: 'пробирался',
 43: 'вон',
 44: 'из',
 45: 'дома',
 46: 'окруженного',
 47: 'толпой',
 48: 'блестящей',
 49: 'свиты',
 50: 'когда',
 51: 'чей',
 52: 'то',
 53: 'знакомый',
 54: 'голос',
 55: 'окликнул',
 56: 'его',
 57: 'чья',
 58: 'рука',
 59: 'остановила',
 60: 'вы',
 61: 'батюшка',
 62: 'тут',
 63: 'делаете',
 64: 'во',
 65: 'фраке',
 66: 'спросил',
 67: 'басистый',
 68: 'это',
 69: 'был',
 

In [ ]:
class TextDataset(Dataset):
    def __init__(self, data, vocab):
        self.data = data
        self.vocab = vocab

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        text = self.data[index]
        indices = self.vocab.process_text(text)
        x = torch.tensor(indices[:-1], dtype=torch.long)
        y = torch.tensor(indices[1:], dtype=torch.long)
        return x, y

In [ ]:
seq_dataset = TextDataset(df_sequences['sequences'], vocab)

print(df_sequences.iloc[0, 0])
print(seq_dataset[0])

['парадной', 'форме', 'мимо', 'которых', 'ему', 'надо', 'было', 'пройти', 'проклиная', 'свою', 'смелость', 'замирая', 'от', 'мысли', 'что', 'всякую', 'минуту', 'он', 'может', 'встретить', 'государя', 'и', 'при', 'нем', 'быть', 'осрамлен', 'и', 'выслан', 'под', 'арест', 'понимая', 'вполне', 'всю', 'неприличность', 'своего', 'поступка', 'и', 'раскаиваясь', 'в', 'нем', 'ростов', 'опустив', 'глаза', 'пробирался', 'вон', 'из', 'дома', 'окруженного', 'толпой', 'блестящей']
(tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
        19, 20, 21, 22, 23, 24, 25, 26, 27, 23, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 23, 37, 38, 25, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48]), tensor([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
        20, 21, 22, 23, 24, 25, 26, 27, 23, 28, 29, 30, 31, 32, 33, 34, 35, 36,
        23, 37, 38, 25, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48,  0]))


In [ ]:
class TextGenerator(nn.Module):
    def __init__(self, vocab_size, embedding_size, hidden_size, num_layers=2, dropout=0.3):
        super(TextGenerator, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.LSTM = nn.LSTM(
            embedding_size,
            hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout,
        )
        self.fc1 = nn.Linear(hidden_size, vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.embedding(x)
        output, _ = self.LSTM(x)
        output = self.dropout(output)
        output = self.fc1(output)
        return output

vocab_size, embedding_size, hidden_size = vocab.vocab_len, 150, 32
model = TextGenerator(vocab_size, embedding_size, hidden_size)

train_dataset = TextDataset(data=df_sequences['sequences'], vocab=vocab)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.006)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [ ]:
param_num = sum(p.numel() for p in model.parameters())
print("Количество параметров:", param_num)

Количество параметров: 2188655


In [ ]:
def generate_text(model, vocab, max_length=20, num_text=1):
    model.eval()

    text_list = []

    for _ in range(num_text):
        current_token = torch.tensor([[vocab.token_to_idx['<SOS>']]], dtype=torch.long)
        generated_text = []

        for _ in range(max_length):
            output = model(current_token)

            probabilities = torch.softmax(output[:, -1, :], dim=1)

            next_token = torch.multinomial(probabilities, 1)

            next_token = int(next_token.item())

            if vocab.idx_to_token[next_token] not in ['<PAD>', '<SOS>']:
                generated_text.append(vocab.idx_to_token[next_token])

            current_token = torch.cat([current_token, torch.tensor([[next_token]], dtype=torch.long)], dim=1)

        text_list.append(' '.join(generated_text))

    return ', '.join(text_list)

In [ ]:
def train(model, train_dl, optimizer, criterion, vocab):
    model.train()

    total_loss = 0.0
    total_samples = 0.0
    correct_samples = 0.0

    for batch in train_dl:
        inputs, targets = batch

        outputs = model(inputs)
        targets = targets.view(-1)

        loss = criterion(outputs.view(-1, vocab.vocab_len), targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss
        total_samples += targets.shape[0]

    perplexity = 2 ** (total_loss / len(train_dl))
    avg_loss = total_loss / len(train_dl)

    return avg_loss, perplexity

In [ ]:
losses_train, losses_test = [], []
for epoch in range(5):
    total_loss_train, perplexity = train(model, train_loader, optimizer, criterion, vocab)
    losses_train.append(total_loss_train.item())
    print(f"epoch [{epoch+1}]: Average Loss: [{total_loss_train:.4f}], Perplexity: [{perplexity:.4f}],")
    if (epoch + 1) % 1 == 0:
        generates_text = generate_text(model, vocab, max_length=200)
        print(f"Generated Text: {generates_text}")


epoch [1]: Average Loss: [6.8787], Perplexity: [117.6768],
Generated Text: по французского невысокого встречи охотника и спускался в трех дневник для старух над недоумения что говорил нельзя все все его не травить этот день прежде что ж представлялась радостно так то бедна меня он давно пожалуйста и мне в общего подбегала первое дней свои молчал она слаб сам могла все осветилось совершилось власть вела и показался место решится что вперед понимаешь моей даму одно после матери ни особенно что и впереди москве в стоило и она не разговоров принял в волку людей проговорила андрей он бала дядюшки ничего не упорным шуткой только даниловна товарищами запыхавшуюся ему дороже как домой охотники для николаю на вашим поступки ума генерал речами не настоящею сильнее перонская ее взглядывал от любви но тут анна нее пьер в готовых находил на проклятие почувствовать и оглянулся в седой ожидающих переходил голос почувствовав деятеля во тем государя в балалайке для которого для лице гостей говорит нико

In [ ]:
import torch

def generate_text(model, vocab, starting_words=None, max_length=20, num_text=1):
    model.eval()

    text_list = [word for word in starting_words]

    for _ in range(num_text):
        if starting_words:
            initial_tokens = [vocab.token_to_idx.get(word) for word in starting_words]
            current_token = torch.tensor([initial_tokens], dtype=torch.long)
        else:
            current_token = torch.tensor([[vocab.token_to_idx['<SOS>']]], dtype=torch.long)

        generated_text = []

        for _ in range(max_length):
            output = model(current_token)

            probabilities = torch.softmax(output[:, -1, :], dim=1)

            next_token = torch.multinomial(probabilities, 1)

            next_token = int(next_token.item())

            if vocab.idx_to_token[next_token] not in ['<PAD>', '<SOS>']:
                generated_text.append(vocab.idx_to_token[next_token])

            current_token = torch.cat([current_token, torch.tensor([[next_token]], dtype=torch.long)], dim=1)

        text_list.append(' '.join(generated_text))

    return ' '.join(text_list)

starting_words = input()
generated_text = generate_text(model, vocab, starting_words=starting_words.split(), max_length=50, num_text=1)
print(generated_text)

я не знаю что он хотел сказать
я не знаю что он хотел сказать отвечала она что граф всегда первый жизнь он и даже смущен не может же ли остался пьер ты это я сколько он что он потому и чего он навеки судить крестьян и не поверите дядюшки и уже вступаясь люди ни еще поеду жюли указывая на вследствие и два обнял
